# CS202: Compiler Construction

## In-class Exercises, Week of 04/17/2023

----

# Select-Instructions

## Question 1

Write an `X86fun` program that's equivalent to the following `Cfun` program:

```
def add1(n):
  add1start:
    tmp_1 = add(n, 1)
    return tmp_1

def main():
  mainstart:
    tmp_2 = add1
    tmp_3 = tmp_2(5)
    print(tmp_3)
    return 0
```

We will do si for each function def seperately, and keep them separate:

```
def add1:
  add1start:
    movq %rdi, #n
    movq %n, #tmp_1
    addq $1, #tmp_1
    movq #tmp_1, %rax
    jmp add1conclusion

def main():
  mainstart:
    --tmp_2 = add1
    leaq add1(%rip), #tmp_2
    --tmp_3 = tmp_2(5)
    movq $5, %rdi
    callq *#tmp_2
    
    movq %rax, #tmp_3
    movq #tmp_3, %rdi
    callq print_int
    
    movq $0, %rax
    call mainconclusion
```

leaq: load effective address
* in callq is an indirect addq

## Question 2

Describe the changes to `select-instructions`.

1. Define a new function `si_def` to compile a function definition
2. New/changed cases: Call, Return, Function assignment (in `si_stmt`)
3. Add a pass-global var called `current_function` to track what function is being compiled (just like explicate-control)

New function `si_def`:
1. Set `current_function` to the name of the function being compiled
2. Call `si_stmts` on the statements of each CFG block
3. For the name + start bock, add statements to set up the arguments: one movq for each parameter, from the paramter passing register to the variables with paramter's name
4. Return a new x86fun function definition

New cases in `si_stmt`:
1. Return should now jump to current_function + conclusion
2. cfun.Assign(x, cfun.Var(f)) if f in function_names use `x86.NamedInstr(`leaq`, [x86.GlobalVal(f), x86.Var(x)])`
3. `cfun.Assign(x, cfun(fun,args))`:
        1. Move arguments into the parameter registers (reverse of the definition case)
        2. Perform an indirect callq: `x86.IndirectCallq(si_atm(fun), 0)`
        3. Move the return value into the destination variable:
            `x86.NamedInstr('movq', [x86.Reg(`rax`), x86.Var(x)])`

----
# Register allocation

## Question 3

Describe the changes to the register allocator.

Copy pass from A6, name it `_allocate_registers`, and call it for each of the functions in the program

This means we perform register allocation for each function seperately.

In addition, change the register allocator to ensure that we follow the calling convention

(Last week Q16: callee-saved vrs caller-saved)

In particular we want to ensure that no value in the caller-saved register will be live across a call to any function.
The Alternative would be to save the caller-saved registers before doing the call (but this requires lots of memory accesses)

Change to `bi_instr` in allocator:
 - for callq and indirectcallq: add an edge in interference graph between each live variable and each caller-saved register (done for us just add indirectcall)

in the To ensure we don't overwrite parameter passing registers before we read in their values:
 - change vars_arg to return registers
 - This ensures that we don't overwrite a paramter-passing register until after we've used trhe value that was in it
 - Effect: put registers we still need in live-after sets of instructions where they might get overwritten

Change initialization of live_before sets:

live_before_sets = {name + `conclusion`: set()}

----
# Patch Instructions

## Question 4

Describe the changes to patch-instructions.

Copy pass from A6, name it `_patch_instructions`, and call it for each of the functions in the program

That's it.

----
# Prelude & Conclusion

## Question 5

Describe the changes to prelude & conclusion.

Copy pass from A6, name it `_prelude_and_conclusion`, and call it for each of the functions in the program

Also need to flatten out function defs into a single x86 program.

Also need to make several other changes:
1. save and restore callee-saved registers:
    - add pushq instructions to prelude to save registers, popq instructions to conclusion to restore them (Hard codable)
2. Initialize the heap only in main function: call the initialize function only if the name is main
3. Add an instruction to tear down root-stack frame
    - x86.NamedInst(subq, [x86.Immediate(8*root_stack_loations), x86.Reg(r15)])
4. Use the function's name as the prelude label
5. Use name + `conclusion` as the conclusion label

----
# Dataclasses and Objects

## Question 6

Transform the following program into an `Ltup` program using tuples.

```
class Point:
    x: int
    y: int

p = Point(1, 2)
v = p.x + p.y
print(v)
```

In [1]:
YOUR ANSWER HERE

3


## Question 7

Describe the changes to the compiler to support dataclasses.

YOUR ANSWER HERE

## Question 8

Transform the following program into an `Lfun` program using tuples.

```
class Point:
    x: int
    y: int
    
    def add(self: Point, other: Point) -> Point:
        return Point(self.x + other.x, self.y + other.y)

p1 = Point(1, 2)
p2 = Point(3, 4)
p3 = p1.add(p2)
v = p3.x + p3.y
print(v)
```

In [3]:
YOUR ANSWER HERE

10
